In [1]:
import sys, random, os, json
sys.path.append(sys.path.append(os.path.join(os.getcwd(), '..')))
from datamart.augment import Augment
import pandas as pd

In [2]:
es_index = "datamart_all"

augment = Augment(es_index=es_index)

### Read a sub set about taxi dataset

In [3]:
old_df = pd.read_csv("./test/taxi.csv")
print(old_df)

    d3mIndex tpep_pickup_datetime  num_pickups
0          0  2018-01-01 00:00:00           67
1          1  2018-01-01 01:00:00            8
2          2  2018-01-01 02:00:00            0
3          3  2018-01-01 03:00:00            0
4          4  2018-01-01 04:00:00            7
5          5  2018-01-01 05:00:00           10
6          6  2018-01-01 06:00:00            9
7          7  2018-01-01 07:00:00           28
8          8  2018-01-01 08:00:00          157
9          9  2018-01-01 09:00:00          259
10        10  2018-01-01 10:00:00          301
11        11  2018-01-01 11:00:00          436
12        12  2018-01-01 12:00:00          369
13        13  2018-01-01 13:00:00          347
14        14  2018-01-01 14:00:00          494
15        15  2018-01-01 15:00:00          544
16        16  2018-01-01 16:00:00          467
17        17  2018-01-01 17:00:00          690
18        18  2018-01-01 18:00:00          461
19        19  2018-01-01 19:00:00          465
20        20 

### Query 1
### Is there any dataset in Datamart has named_entity new york and is related to wind?

In [4]:
metadatas = augment.query_by_key_value_pairs(key_value_pairs=[
    ("variables.named_entity", "new york"),
    ("description", "wind")
])
print(len(metadatas))


17


### Query 2
### Is there any dataset in Datamart has temporal coverage and it covers the range between `start` and `end`

In [5]:
metadatas_satisfy_temporal = augment.query_by_temporal_coverage(
    start="2018-01-01 00:00:00",
    end="2018-01-03 00:00:00"
)
print(len(metadatas_satisfy_temporal))


12267


### Get the intersection of Query 1 and 2. Print datamart_id

In [6]:
intersections = augment.get_metadata_intersection(metadatas, metadatas_satisfy_temporal)
print([x["_source"]["datamart_id"] for x in intersections])

[125480000, 124480000, 124620000, 124600000, 125530000, 124530000, 125150000, 125450000, 125030000]


### Materialize them with their metadata. Constrain should come from UI, will have UI for user to form such constrain.

In [7]:
for hitted in intersections:
    
    named_entity_column = None
    for idx, variable in enumerate(hitted["_source"]["variables"]):
        if variable.get("named_entity", None):
            named_entity_column = idx
            break
    
    df = augment.get_dataset(
        metadata=hitted["_source"],
        constrains={
            "named_entity": {
                named_entity_column: ["new york"]
            },
            "date_range": {
                "start": "2018-01-01T00:00:00",
                "end": "2018-01-02T23:00:00"
            }
        }
    )
    print("========{}========".format(hitted["_source"]["description"]))
    print(df)
    print("\n\n")


========Direction of fastest 2-minute wind (degrees)========
                   date          stationid      city  WDF2
0   2018-01-01T00:00:00  GHCND:USW00014732  new york   320
1   2018-01-01T00:00:00  GHCND:USW00014734  new york   330
2   2018-01-01T00:00:00  GHCND:USW00054743  new york   320
3   2018-01-01T00:00:00  GHCND:USW00054787  new york   330
4   2018-01-01T00:00:00  GHCND:USW00094728  new york   300
5   2018-01-01T00:00:00  GHCND:USW00094741  new york   310
6   2018-01-01T00:00:00  GHCND:USW00094745  new york   290
7   2018-01-01T00:00:00  GHCND:USW00094789  new york   310
8   2018-01-02T00:00:00  GHCND:USW00014732  new york   270
9   2018-01-02T00:00:00  GHCND:USW00014734  new york   260
10  2018-01-02T00:00:00  GHCND:USW00054743  new york   290
11  2018-01-02T00:00:00  GHCND:USW00054787  new york   250
12  2018-01-02T00:00:00  GHCND:USW00094728  new york   260
13  2018-01-02T00:00:00  GHCND:USW00094741  new york   300
14  2018-01-02T00:00:00  GHCND:USW00094745  new york  

========Peak gust wind speed (tenths of meters per second)========
Empty DataFrame
Columns: [date, stationid, city, WSFG]
Index: []





========Direction of fastest 5-second wind (degrees)========
                   date          stationid      city  WDF5
0   2018-01-01T00:00:00  GHCND:USW00014732  new york   300
1   2018-01-01T00:00:00  GHCND:USW00014734  new york   320
2   2018-01-01T00:00:00  GHCND:USW00054743  new york   320
3   2018-01-01T00:00:00  GHCND:USW00054787  new york   310
4   2018-01-01T00:00:00  GHCND:USW00094728  new york   300
5   2018-01-01T00:00:00  GHCND:USW00094741  new york   320
6   2018-01-01T00:00:00  GHCND:USW00094745  new york   310
7   2018-01-01T00:00:00  GHCND:USW00094789  new york   310
8   2018-01-02T00:00:00  GHCND:USW00014732  new york   270
9   2018-01-02T00:00:00  GHCND:USW00014734  new york   270
10  2018-01-02T00:00:00  GHCND:USW00054743  new york   290
11  2018-01-02T00:00:00  GHCND:USW00054787  new york   300
12  2018-01-02T00:00:00  GHCND:USW00094728  new york   250
13  2018-01-02T00:00:00  GHCND:USW00094741  new york   290
14  2018-01-02T00:00:00  GHCND:USW00094745  new york  

========Fastest 5-second wind speed (tenths of meters per second)========
                   date          stationid      city  WSF5
0   2018-01-01T00:00:00  GHCND:USW00014732  new york   143
1   2018-01-01T00:00:00  GHCND:USW00014734  new york   143
2   2018-01-01T00:00:00  GHCND:USW00054743  new york   112
3   2018-01-01T00:00:00  GHCND:USW00054787  new york   103
4   2018-01-01T00:00:00  GHCND:USW00094728  new york   112
5   2018-01-01T00:00:00  GHCND:USW00094741  new york   134
6   2018-01-01T00:00:00  GHCND:USW00094745  new york   134
7   2018-01-01T00:00:00  GHCND:USW00094789  new york   152
8   2018-01-02T00:00:00  GHCND:USW00014732  new york   139
9   2018-01-02T00:00:00  GHCND:USW00014734  new york   116
10  2018-01-02T00:00:00  GHCND:USW00054743  new york    98
11  2018-01-02T00:00:00  GHCND:USW00054787  new york   107
12  2018-01-02T00:00:00  GHCND:USW00094728  new york   125
13  2018-01-02T00:00:00  GHCND:USW00094741  new york   116
14  2018-01-02T00:00:00  GHCND:USW0009474

========24-hour wind movement (km)========
Empty DataFrame
Columns: [date, stationid, city, WDMV]
Index: []





========Direction of peak wind gust (degrees)========
Empty DataFrame
Columns: [date, stationid, city, WDFG]
Index: []





========Average daily wind speed (tenths of meters per second)========
                   date          stationid      city  AWND
0   2018-01-01T00:00:00  GHCND:USW00014732  new york    76
1   2018-01-01T00:00:00  GHCND:USW00014734  new york    57
2   2018-01-01T00:00:00  GHCND:USW00054743  new york    29
3   2018-01-01T00:00:00  GHCND:USW00054787  new york    38
4   2018-01-01T00:00:00  GHCND:USW00094728  new york    35
5   2018-01-01T00:00:00  GHCND:USW00094741  new york    56
6   2018-01-01T00:00:00  GHCND:USW00094745  new york    57
7   2018-01-01T00:00:00  GHCND:USW00094789  new york    79
8   2018-01-02T00:00:00  GHCND:USW00014732  new york    62
9   2018-01-02T00:00:00  GHCND:USW00014734  new york    64
10  2018-01-02T00:00:00  GHCND:USW00054743  new york    31
11  2018-01-02T00:00:00  GHCND:USW00054787  new york    48
12  2018-01-02T00:00:00  GHCND:USW00094728  new york    36
13  2018-01-02T00:00:00  GHCND:USW00094741  new york    50
14  2018-01-02T00:00:00  GHCND:USW00094745  

========Fastest 2-minute wind speed (tenths of meters per second)========
                   date          stationid      city  WSF2
0   2018-01-01T00:00:00  GHCND:USW00014732  new york   112
1   2018-01-01T00:00:00  GHCND:USW00014734  new york   107
2   2018-01-01T00:00:00  GHCND:USW00054743  new york    76
3   2018-01-01T00:00:00  GHCND:USW00054787  new york    72
4   2018-01-01T00:00:00  GHCND:USW00094728  new york    67
5   2018-01-01T00:00:00  GHCND:USW00094741  new york    94
6   2018-01-01T00:00:00  GHCND:USW00094745  new york    98
7   2018-01-01T00:00:00  GHCND:USW00094789  new york   130
8   2018-01-02T00:00:00  GHCND:USW00014732  new york    98
9   2018-01-02T00:00:00  GHCND:USW00014734  new york    94
10  2018-01-02T00:00:00  GHCND:USW00054743  new york    76
11  2018-01-02T00:00:00  GHCND:USW00054787  new york    76
12  2018-01-02T00:00:00  GHCND:USW00094728  new york    72
13  2018-01-02T00:00:00  GHCND:USW00094741  new york    89
14  2018-01-02T00:00:00  GHCND:USW0009474

========Highest instantaneous wind speed (tenths of meters per second)========
Empty DataFrame
Columns: [date, stationid, city, WSFI]
Index: []





### We do get some datasets related to `wind` from NOAA, how to join with old dataframe?